In [1]:
from reinforcement_training import *
from notebook_game_helper import *

Using Theano backend.


In [2]:
training_game_size_x = 40
training_game_size_y = 30

layers = 3
neurons = 120
layer = {"size":neurons,"activation":"tanh"}
hiddens = [layer for i in range(layers)]    # make an optimizer
from keras.optimizers import sgd, RMSprop, Adagrad, Adadelta, Adam
# note to self: DON'T CHANGE THIS UNTIL YOU KNOW WE'RE LEARNING SOMETHING
# optimizer = sgd(lr = 0.0001)
# optimizer_str = "SGD"
# optimizer = Adagrad()
# optimizer_str = "Adagrad"
# optimizer = RMSprop()
# optimizer_str = "RMSprop"
optimizer = Adadelta()
optimizer_str = "Adadelta"
# optimizer = Adam()
# optimizer_str = "Adam"
model = baseline_model(optimizer, hiddens)
# prepare the game for training model
training_game = ReinforcementNaviGame(training_game_size_y,
                                training_game_size_x,
                                model,
                                tolerance = 10)
training_game.setup()

In [ ]:
animate_game(training_game, 10)

In [ ]:
training_game.Navigator.move(x = 19, y = 14)

In [ ]:
training_episodes = 5000
steps = 10

output = train_model(game = training_game,
                model = model,
                episodes = training_episodes,
                steps = steps)

Episode 1781, Replay Loss: 0.0839:  36%|███▌      | 1782/5000 [02:38<05:01, 10.66it/s]]]

In [ ]:
# plot learning info
title_str = str(training_game_size_y) + "x" + str(training_game_size_x) + " with "
title_str += str(training_episodes) + " episodes, " + str(steps) + " steps per episode, & "
title_str += str(len(hiddens)) + " hidden layers, optimized with " + optimizer_str + "\n"
f, axarr = pl.subplots(3, 1, figsize = (10, 15), dpi = 300)
# f.canvas.set_window_title("RL Loss, 100 eps w/ 50 steps, Look: 20")
mean_step = 10
num_means = int(len(output['distances'])/mean_step/steps)

for _, k in enumerate([10, 100, 500]):
    mean_step = k
    mean_rewards = []
    mean_dists = []
    mean_loss = []
    num_means = int(len(output['distances'])/mean_step/steps)
    steps_per_mean = steps*mean_step
    x = np.linspace(0, training_episodes, num_means)
    for i in range(num_means):
        mean_r = 0
        mean_d = 0
        mean_l = 0
        for j in range(steps_per_mean):
            mean_r += output['rewards'][j + i * steps_per_mean]
            mean_d += output['distances'][j + i * steps_per_mean]
            mean_l += output['loss'][j + i * steps_per_mean]
        mean_r = mean_r / steps_per_mean
        mean_d = mean_d / steps_per_mean
        mean_l = mean_l / steps_per_mean
        mean_rewards.append(mean_r)
        mean_dists.append(mean_d)
        mean_loss.append(mean_l)
    label = str(mean_step) + " Episodes"
    axarr[0].plot(x, mean_loss, label = label)
    axarr[1].plot(x, mean_dists, label = label)
    axarr[2].plot(x, mean_rewards, label = label)

axarr[0].grid(True)
axarr[0].set_title(title_str + 'Mean Loss')
axarr[1].grid(True)
axarr[1].set_title('Mean Distances from Goal')
axarr[2].grid(True)
axarr[2].set_title('Mean Rewards')
f.subplots_adjust(hspace=0.2)

file_str = str(training_game_size_y) + "x" + str(training_game_size_x) + "_"
file_str += str(training_episodes) + "_" + str(steps) + "_" + str(len(hiddens))
file_str += "_" + str(neurons) + "_neurons_"+ optimizer_str
pl.legend()
pl.plot()
# pl.savefig("rl_plots" + file_str + ".png")
pl.show()
